<a href="https://colab.research.google.com/github/bradymiller2310/FantasyFootballDashboard/blob/main/ESPN_FF_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install espn-api
!pip install pandas
!pip install openpyxl

In [ ]:
from espn_api.football import League
import pandas as pd

In [ ]:
league = League(league_id=1292514, year=2024, espn_s2="AECPNWyLvfnlcFfE0Xb73O%2BP5ZU9n1G%2FSnP3ixxvuqSwQRcQe7bEQfBuz3YBNeRluTYofdhX2aTajj0jZSF%2B8i9ewdk8Dwf6p9G215F2FjmI06wPKyhjleL4WzaT3PCiP15FDXe55YNMkQeJXgbsCuK1VHbdQ1Nxidq0JxAHezRn2zAt6WwNtSqrF0k2nNHV33VpJqPmMk4msFmaoCaz0UAp95xxy1YmXyEUnZWgA60HJkbsgYkZ8AaESwFENsPYdUvKYqCwnZLjvgI5varhOTDYekjvokdtaCoxJfzOdGLJ3g%3D%3D", swid="{D68FE5F2-46CE-421C-B248-DDA01360BB34}")
print(league.teams)  #Teams in league

[Team(Fightin' Furries), Team(Quon Solo), Team(Jen-eral ⚔️), Team(Captain Sweatpants), Team(Salmon LipBalm !!), Team(Graham’s Groupie), Team(pop-pop's bible study), Team(Bucktown Bandits), Team(bird gang), Team(bungalicious  💅)]


In [ ]:
for week in range(1, 15):
    print(f"Week {week} Schedule:")
    for matchup in league.scoreboard(week=week):
        print(f"{matchup.home_team} vs {matchup.away_team}")

In [ ]:
free_agents = league.free_agents(size=20, position='QB')  # Get top 20 available QBs

print("Available Free Agents (QB):")
for player in free_agents:
    print(f"  - {player.name} ({player.proTeam})")

In [ ]:
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    print("Roster:")
    for player in team.roster:
        print(f"  - {player.name} ({player.position})")

In [ ]:
week_number = 1
box_scores = league.box_scores(week=week_number)

for matchup in box_scores:
    print(f"\nMatchup: {matchup.home_team} vs {matchup.away_team}")
    print(f"  Home Score: {matchup.home_score}")
    print(f"  Away Score: {matchup.away_score}")

In [ ]:
# getting injury status of free agents (not on rosters)

free_agents = league.free_agents()
i=0
for player in free_agents:
  player_obj = free_agents[i]
  print(f"Player: {player_obj.name}, Injury Status: {player_obj.injuryStatus}, Injured: {player_obj.injured}")
  i+=1

In [ ]:
# getting injury status of players on rosters
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    for player in team.roster:
        print(f"Player: {player.name}, Injury Status: {player.injuryStatus}, Injured: {player.injured}")

In [ ]:
# getting starting roster for each week

for team in league.teams:
    print(f"\nTeam: {team.team_name} (Week {week})")

    # Get the team's roster for the given week
    for player in team.roster:
        #if player.lineupSlot != "BE" and player.lineupSlot != "IR":  # Exclude Bench and IR players
            print(f"  - {player.name} ({player.lineupSlot})")


In [ ]:
roster_data = []

# Loop through each team in the league
for team in league.teams:
    for player in team.roster:
        # Collect the relevant data
        roster_data.append({
            "player_id": player.playerId,  # ESPN Player ID
            "player_name": player.name,
            "fantasy_team": team.team_name,
            "roster_slot": player.lineupSlot  # e.g., "RB", "WR", "BE"
        })

# Convert to DataFrame
df = pd.DataFrame(roster_data)

# Display the DataFrame
print(df)

### **Getting statistical data from xlsx files**

In [ ]:
from openpyxl import load_workbook
import pandas as pd

def safe_read_sheet(file_path, sheet_name):
    wb = load_workbook(filename=file_path, read_only=True, data_only=True)
    ws = wb[sheet_name]

    data = []
    for row in ws.iter_rows(values_only=True):
        if any(cell is not None for cell in row):  # Skip fully empty rows
            data.append(row)

    df = pd.DataFrame(data)

    # Promote first row to header
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    # Convert all values to string (mimicking dtype=str)
    return df.astype(str)

# Use your actual file name here
file_path = "ff_data.xlsx"  # Adjust if needed

# If you don't know sheet names:
wb = load_workbook(filename=file_path, read_only=True)
print("Sheets available:", wb.sheetnames)

# Then try loading a sheet
df = safe_read_sheet(file_path, sheet_name="Season")  # or "Weekly", etc.
df.head()

TypeError: expected <class 'int'>

In [31]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [35]:
import pandas as pd

file_path = "/content/drive/My Drive/ff_data.xlsx"

# Load specific sheets
season_stats_all = pd.read_excel(file_path, sheet_name="Season", engine="openpyxl")
weekly_stats_all = pd.read_excel(file_path, sheet_name="Weekly", engine="openpyxl")

# Display first few rows
print(season_stats_all.head())
print(weekly_stats_all.head())

   season   player_id player_display_name team position  games_played  \
0    2021  00-0019596           Tom Brady   TB       QB          20.0   
1    2021  00-0022924  Ben Roethlisberger  PIT       QB          18.0   
2    2021  00-0023459       Aaron Rodgers   GB       QB          17.0   
3    2021  00-0023682    Ryan Fitzpatrick  WAS       QB           1.0   
4    2021  00-0024243      Marcedes Lewis   GB       TE          14.0   

   completions  attempts  comp%  passing_yards  ...  total_st_tds  st_tds_pg  \
0        573.0     847.0  67.65         6187.0  ...           NaN        NaN   
1        448.0     693.0  64.65         4170.0  ...           NaN        NaN   
2        386.0     560.0  68.93         4340.0  ...           NaN        NaN   
3          3.0       6.0  50.00           13.0  ...           NaN        NaN   
4          0.0       0.0    NaN            0.0  ...           NaN        NaN   

   total_fp  fp_pg  espn_id                name   age  height  weight  \
0      

In [ ]:
# Define the range of weeks
weeks = range(1, 19)  # Weeks 1 to 18

# Create an empty list to store data
all_weeks_data = []

# Loop through each week and get the roster data
for week in weeks:
    week_data = []  # Temporary storage for current week's data

    for team in league.teams:
        team_name = team.team_name  # Fantasy Team Name

        for player in team.roster:
            week_data.append({
                "Team Name": team_name,
                "Week": week,
                "Player Name": player.name,
                "ESPN ID": player.playerId,  # ESPN Unique Player ID
                "Position": player.position if player.lineupSlot != "BE" else "Bench"
            })

    # If no data is returned for this week, stop iterating
    if not week_data:
        print(f"No data found for Week {week}. Stopping iteration.")
        break  # Stop looping when a week has no data

    # Add this week's data to the master list
    all_weeks_data.extend(week_data)

# Convert to DataFrame
df = pd.DataFrame(all_weeks_data)


In [ ]:
print(df.head())

          Team Name  Week      Player Name  ESPN ID Position
0  Fightin' Furries     1      Tyreek Hill  3116406       WR
1  Fightin' Furries     1    Isiah Pacheco  4361529       RB
2  Fightin' Furries     1        Joe Mixon  3116385    Bench
3  Fightin' Furries     1  Patrick Mahomes  3139477       QB
4  Fightin' Furries     1   Dalton Kincaid  4385690    Bench


In [ ]:
full_season_data = pd.merge(df, season_stats_all, left_on="ESPN ID", right_on = "espn_id", how="left")
full_weekly_data = pd.merge(df, weekly_stats_all, left_on="ESPN ID", right_on = "espn_id", how="left")

***Currently have data all ready to go besides defensive scoring - still need to figure out adding defenseive touchdowns to their scoring. But we have all the data for the model***